##### **Quick overview**

Part 2 of this project aimed to look at the pills data up close, particularly into the dosage forms, colours, shapes and inactive excipients used in the dataset. Plotly was used as the main data visualisation library here, with a section in the end to generate a small dataframe in preparation for a data visualisation in Rust-Evcxr for part 3.

<br>

##### **Import libraries and pills dataset**


In [ ]:
import polars as pl
import plotly.express as px

The pills.csv file saved from part 1 was imported as shown below.


In [ ]:
df = pl.read_csv("pills.csv")
df

<br>

##### **Change column names**

Again, column names were changed to something easier to read.


In [ ]:
# Rename all column names
df_new = df.rename({"splcolor_text": "Colour", 
                    "splshape_text": "Shape", 
                    "spl_strength": "Drug_strength", 
                    "spl_inactive_ing": "Inactive_excipients", 
                    "DosageForm": "Dosage_form"}
                  )
df_new

<br>

##### **Visualising oral dosage forms & colours in pills**

Grabbing only unique drugs in the dataset to minimise duplications.


In [ ]:
df_viz = df_new.unique(subset = "Drug_strength")
df_viz

One way to avoid switching Polars dataframe to a Pandas one, which might be one of the options to plot data from Polars dataframes in Plotly, was to call the x-axis and y-axis datasets directly as shown in the codes below.


In [ ]:
# scatter plot for colours, dosage forms & drug strengths 
fig = px.scatter(x = df_viz["Colour"], 
                 y = df_viz["Dosage_form"], 
                 color = df_viz["Colour"],
                 hover_name = df_viz["Drug_strength"],
                 width = 900, 
                 height = 400,
                 title = "Oral dosage forms and colours of pills")

fig.update_layout(
    title = dict(
        font = dict(
            size = 15)),
    title_x = 0.5,
    margin = dict(
        l = 20, r = 20, t = 40, b = 3),
    xaxis = dict(
        tickfont = dict(size = 9), 
        title = "Colours"
    ),
    yaxis = dict(
        tickfont = dict(size = 9), 
        title = "Dosage forms"
    ),
    legend = dict(
        font = dict(
            size = 9)))

fig.show()

White was the most common colour, especially after zooming in the plot. Capsule was very commonly used as the oral dosage form of choice in this dataset.

<br>

##### **Visualising shapes & colours in pills**


In [ ]:
fig = px.scatter(x = df_viz["Colour"], 
                 y = df_viz["Shape"], 
                 color = df_viz["Colour"],
                 hover_name = df_viz["Drug_strength"],
                 width = 900, 
                 height = 400,
                 title = "Shapes and colours of pills")

# Update layout of the plot
fig.update_layout(
    # Change title font size
    title = dict(
        font = dict(
            size = 15)),
    # Centre the title
    title_x = 0.5,
    # Edit margins
    margin = dict(
        l = 20, r = 20, t = 40, b = 3),
    # Change x-axis
    xaxis = dict(
        tickfont = dict(size = 9), 
        title = "Colours"
    ),
    # Change y-axis
    yaxis = dict(
        tickfont = dict(size = 9), 
        title = "Shapes"
    ),
    # Edit lengend font size
    legend = dict(
        font = dict(
            size = 9)))

fig.show()

Capsule was again the most common oral dosage shape used for pills in the dataset. Common colours included red, brown, blue, purple, pink, orange, green, white and yellow. Combination colours followed these common ones, which had a mixture of a variety of colours used simultaneously, likely to avoid confusions and errors in dispensings or administrations.

<br>

##### **Visualising inactive excipients in pills**

The messiest part of the data actually lied in the column of "Inactive_excipients", with numerous different punctuations used inconsistently, such as forward slashes, commas and semi-colons. There were vast quantities of different inactive components used for oral dosage forms. Because of this, I had to spend a bit more time cleaning up the texts in order to observe what sort of visulaisations would result in the end.


In [ ]:
# Formulated a separate dataframe with just "Inactive_excipients"
df_ie = df_new.select([pl.col("Inactive_excipients")])
df_ie

###### **Text cleaning for inactive excipients column**

In order to prepare this column for data visualisations, I used Polars' string expressions to try and tidy up the raw texts. When I did the text cleaning in Jupyter Lab initially, the line of code for .str.strip(" ,") worked, but when I converted the .ipynb file into a .qmd (Quarto markdown) one, and used the same line, it failed to work due to the extra space in front of the comma. However, I got around the error by splitting it into two separate units as space and comma, and it worked without problem. One possible reason would be due to the reticulate package needed to run Python in RStudio IDE, and how Polars dataframe library was relatively newer than Pandas dataframe library, which meant certain features in Polars might not have been taken on board in the reticulate package (only my guess).


In [ ]:
# Clean string texts
df_de = (df_ie.with_column(pl.col("Inactive_excipients").str.to_lowercase(
    ).str.replace_all(
        ";", ", "
    ).str.replace_all(
        " /", ", "
    ).str.replace_all(
        "/", ", "
    # In Jupyter notebook/lab: .str.strip(" ,") - combine space & comma
    ).str.strip(
        " "
    ).str.strip(
        ","
    ).str.split(
        by = ", "
    ).alias(
        "Inactive"
    )
).explode(
    "Inactive"
)
)

df_de

In [ ]:
# Quick look at the dataframe to see before and after text cleaning
print(df_de.glimpse())

As shown above, the "Inactive_excipients" column was the original column for excipients, where the second column named, "Inactive" was the new column shown after the first string strip and row text explosion. The excipients were broken down into smaller units than previously.


In [ ]:
# Re-organise the dataframe to choose the cleaned "Inactive" column
df_final = df_de.select(["Inactive"])
df_final

In [ ]:
# Remove all cells with null values
df_final = df_final.drop_nulls()

In [ ]:
# Group the data by different inactive excipients with counts shown
df_final = df_final.groupby("Inactive").agg(pl.count())
df_final.head()

###### **Inactive excipient counts**


In [ ]:
# Count each excipient and cast the whole column into integers
df_final = df_final.with_column((pl.col("count")).cast(pl.Int64, strict = False))
df_final

###### **Overview of inactive excipients used in oral dosage forms**


In [ ]:
fig = px.scatter(x = df_final["Inactive"], 
                 y = df_final["count"], 
                 hover_name = df_final["Inactive"],
                 title = "Inactive excipients and their respective counts in pills")

fig.update_layout(
    title = dict(
        font = dict(
            size = 15)),
    title_x = 0.5,
    margin = dict(
        l = 20, r = 20, t = 40, b = 10),
    xaxis = dict(
        tickfont = dict(size = 9), 
        title = "Inactive excipients"
    ),
    yaxis = dict(
        tickfont = dict(size = 9), 
        title = "Counts"
    ),
    legend = dict(
        font = dict(
            size = 9)))


fig.show()

###### **Frequently used inactive excipients**


In [ ]:
# Re-order the excipients with counts in descending order
# Filter out only the ones with counts over 10,000
df_ex = df_final.sort("count", reverse = True).filter((pl.col("count")) >= 10000)
df_ex.head()

In [ ]:
fig = px.bar(x = df_ex["Inactive"], 
             y = df_ex["count"], 
             color = df_ex["Inactive"],
             title = "Commonly used inactive excipients in pills")

fig.update_layout(
    title = dict(
        font = dict(
            size = 15)),
    title_x = 0.5,
    margin = dict(
        l = 10, r = 10, t = 40, b = 5),
    xaxis = dict(
        tickfont = dict(size = 9), 
        title = "Inactive excipients"
    ),
    yaxis = dict(
        tickfont = dict(size = 9), 
        title = "Counts"
    ),
    legend = dict(
        font = dict(
            size = 9)))

fig.show()

<br>

##### **Create a small dataframe for data visualisation in Rust-Evcxr**

Filtered out all acetaminophens in the "Drug_strength" column and removed all duplicates in the dataset.


In [ ]:
df_ac = df_new.filter(
    pl.col("Drug_strength")
    .str.starts_with("acetam")).unique(subset = ["Drug_strength"])

df_ac

In [ ]:
df_ac = df_ac.groupby("Colour").agg(pl.count())
df_ac

In [ ]:
fig = px.scatter(x = df_ac["Colour"], 
                 y = df_ac["count"], 
                 size = df_ac["count"], 
                 color = df_ac["Colour"],
                 title = "Frequency of colours in acetaminophen (paracetamol) oral dosage forms"
                )

fig.update_layout(
    xaxis = dict(
        title = "Colours"
    ), 
    yaxis = dict(
        title = "Counts"
    )
)

fig.show()

I've decided to keep the dataframe very simple for part 3 as my original intention was to trial plotting a graph in Evcxr only (nothing fancy at this stage), and also to gain some familiarities with Rust as another new programming language for me. Readers might notice that I've opted for a scatter plot here for this last dataframe, and when we finally got to part 3 (hopefully coming soon as I needed to figure how to publish Rust codes in Quarto...), I might very likely revert this to a bar graph, due to some technical issues and being a new Rust-Evcxr user. At the very least, I've kind of tried something I've planned for, although it might not look very elegant yet.